In [ ]:
!pip install pandas
!pip install networkx
!pip install scipy 

In [ ]:
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

with open('systems.json') as f:  
    data = json.load(f)
df=pd.DataFrame(data)

with open('planets.json') as f:  
    data = json.load(f)
df_planets=pd.DataFrame(data)
#print(df)

#for row in range(len(df)):
#  print(df.at[row, 'Name'])

In [ ]:
#df.dtypes
#df[1:4]
df

In [ ]:
print(df_planets.dtypes)
# df_planets

In [ ]:
df_planets["COGCPrograms"].loc[4150]

In [ ]:
id_to_name = {}
for index, row in df.iterrows():
    id_to_name[row["SystemId"]] = row["Name"]

In [ ]:


def FindSystem(name):
    for index, row in df.iterrows():
        if row["Name"] == name: #weird type stuff????
            return row # returns a dataframe, I think?

def FindPlanetsIn(systemDf):
    matches = []
    for index, row in df_planets.iterrows():
        if id_to_name[row["SystemId"]] == systemDf["Name"]:
            matches.append(row)
    return matches
        
ANT_node = "Antares I"
BEN_node = "Benten"
MOR_node = "Moria"

fullGraph = nx.Graph()
#G.from_pandas_dataframe(df)
#list(G.nodes)

# fill graph object with nodes
for index, row in df.iterrows():
    fullGraph.add_node(row["Name"])

# and connections
for index, row in df.iterrows():
    for connection in row["Connections"]:
        #print(row["Name"] + " to " + id_to_name[connection["ConnectingId"]])
        fullGraph.add_edge(row["Name"], id_to_name[connection["ConnectingId"]])
    
print("Node count " + str(fullGraph.number_of_nodes()) + " edge count: " + str(fullGraph.number_of_edges()))

# maybe we are only interested in the systems near ANT

# within N jumps of ANT
def filterWithin15ANT(systemName):
    pathLen = nx.shortest_path_length(fullGraph, ANT_node, systemName)
    # print("Distance from " + systemName + " is " + str(pathLen))
    return pathLen < 5

# planets closer to ANT than they are to other CXes
def filterCloserToANT(systemName):
    distANT = nx.shortest_path_length(fullGraph, ANT_node, systemName)
    distBEN = nx.shortest_path_length(fullGraph, BEN_node, systemName)
    distMOR = nx.shortest_path_length(fullGraphG, MOR_node, systemName)
    # print("Distance from " + systemName + " is " + str(pathLen))
    return distANT <= distBEN and distANT <= distMOR

# all planets
def filterAll(systemName):
    return True


def FilterByFunction(filterPlanetFunction):
    selected_nodes = []
    for index, row in df.iterrows():
        systemName = row["Name"]
        if(filterPlanetFunction(systemName)):
            selected_nodes.append(systemName)
        
#     print("~~~~~~~~~~~~~~~~~~~")
    subGraph = fullGraph.subgraph(selected_nodes)
#     print("after filter: Node count " + str(subGraph.number_of_nodes()) + " edge count: " + str(subGraph.number_of_edges()))
    return subGraph

In [ ]:
# find game specified xy positions:
game_pos = {}
for index, row in df.iterrows():
    if row["Name"] in fullGraph.nodes():
#         print("Find pos for " + row["Name"])
        game_pos[row["Name"]] = [row["PositionX"] / 1000.0 , row["PositionY"] / 1000.0]

In [ ]:
# find systems with nice planets.
filteredValue = {}
filteredValue2 = {}
filteredValue3 = {}

for index, row in df_planets.iterrows():
    
    systemName = id_to_name[row["SystemId"]]
    planetName = row["PlanetName"]

    filteredValue.setdefault(systemName, 0)
    filteredValue2.setdefault(systemName, 0)
    filteredValue3.setdefault(systemName, 0)

#     if(row["Fertility"] > -1):
#         print("Fertile: " + str(row["Fertility"]) + " on " + planetName + " in " + systemName)
#         filteredValue[systemName] = filteredValue.get(systemName, 0) + 1

    malusCount = 0
    pressure = row["Pressure"]
    temperature = row["Temperature"]
    gravity = row["Gravity"]

    if(gravity > 2.5 or gravity < 0.25):
        malusCount += 1
    if(temperature > 75 or temperature < -25):
        malusCount += 1
    # low pressure is fine, so or pressure < 0.25 is okay!
#    if(pressure > 2.0 or pressure < 0.25 ):
#        malusCount += 1
    if(pressure > 2.0 ):
        malusCount += 1

    if(malusCount == 0):
        filteredValue[systemName] = filteredValue.get(systemName, 0) + 1
        #print(str(malusCount) + " for pressure " + str(pressure) + " on " + planetName)

    #filteredValue = dict(nx.single_source_shortest_path_length(G, favoredNode))

    for reso in row["Resources"]:
        #print("Planet " + planetName + " has " + str(reso))
        #water
        if reso["MaterialId"] == "ec8dbb1d3f51d89c61b6f58fdd64a7f0" \
            and reso["ResourceType"] == "LIQUID" \
            and reso["Factor"] > 0.36: 
            # ResourceType = LIQUID Factor > 0.36 (Harmonia levels)
            filteredValue2[systemName] = 1
            #print("Found water on " + planetName)


    #consider it occupied if it has a COGC
    #print(planetName + " in " + systemName + " has COGC: " + str(row["HasChamberOfCommerce"]))
    if row["HasChamberOfCommerce"]:
        filteredValue3[systemName] = filteredValue3.get(systemName, 0) + 1
        #print("Set!")


In [ ]:
filteredValue3

In [ ]:

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def PlotGraph(G):
    # pos = nx.spring_layout(G, iterations=100, seed=173, scale = 2)
    # pos = nx.spiral_layout(G)
    # pos = nx.shell_layout(G)
    #pos = game_pos
    pos = nx.kamada_kawai_layout(G, pos=game_pos)

    plt.clf()
    plt.figure(figsize=(35,35))

    print("Drawing: Node count " + str(G.number_of_nodes()) + " edge count: " + str(G.number_of_edges()))

    # draw extra nodes to create outlines and dots inside our real nodes.
    matchingKeys = [k for k,v in filteredValue3.items() if float(v) >= 1]
    f1 = intersection(G.nodes, list(matchingKeys))
    nx.draw_networkx_nodes(G, pos, node_size=210, node_color='#00FF00', nodelist=f1)

    f1 = intersection(G.nodes, list(filteredValue.keys()))
    colorListValues = [filteredValue[k] for k in f1]
    nx.draw(G, pos, node_size=150, with_labels=False, nodelist=f1, node_color=colorListValues, cmap="inferno")

    # blue nodes
    matchingKeys = [k for k,v in filteredValue2.items() if float(v) >= 1]
    f1 = intersection(G.nodes, list(matchingKeys))
    nx.draw_networkx_nodes(G, pos, node_size=20, node_color='#0000FF', nodelist=f1)


    label_ratio = 1.0/16.0
    pos_labels = {}
    for aNode in G.nodes():
        #Get the node's position from the layout
        x,y = pos[aNode]
        #Get the node's neighbourhood
        N = G[aNode]
        #Find the centroid of the neighbourhood. The centroid is the average of the Neighbourhood's node's x and y coordinates respectively.
        #Please note: This could be optimised further
        cx = sum(map(lambda x:pos[x][0], N)) / len(pos)
        cy = sum(map(lambda x:pos[x][1], N)) / len(pos)
        #Get the centroid's 'direction' or 'slope'. That is, the direction TOWARDS the centroid FROM aNode.
        slopeY = (y-cy)
        slopeX = (x-cx)
        #Position the label at some distance along this line. Here, the label is positioned at about 1/8th of the distance.
        pos_labels[aNode] = [x+slopeX*label_ratio, y+slopeY*label_ratio]

        #ignore all the math we did above and just offset it from the planet. 
        pos_labels[aNode] = [x + 0.01, y + 0.01]


    # pos = pos_labels
    #  "ec": "k",
    label_options = { "fc": "white", "alpha": 0.2}
    nx.draw_networkx_labels(G, pos_labels, font_size=7, bbox=label_options)

    s = "Green outline indicates COGC. Blue dot indicates usable quantities of water ( liquid and > Harmonia ). Color indicates number of planets that don't need a building material other than SEA or MCG"
    plt.title("Prosperous Universe\nmap by Archiel\n" + s)
    # plt.text(0.5, 0.98, s, horizontalalignment="center")
    plt.tight_layout()
    plt.show()

In [ ]:
intersection(FilterByFunction(filterAll), list(filteredValue2.keys()))

In [ ]:

  
#graph = FilterByFunction(filterWithin15ANT)
graph = FilterByFunction(filterAll)
PlotGraph(graph)


In [ ]:

def to_camel_case(text):
    return text[0]+text.title()[1:].replace("-"," ").replace("_"," ")

def EvaluatePlanet(planetDf):
    pressure = planetDf["Pressure"]
    temperature = planetDf["Temperature"]
    gravity = planetDf["Gravity"]
    surface = planetDf["Surface"]

    cost = 1688
    note = ""
    if gravity > 2.5: 
        cost += 85
        note += "High Grav (BL), "
    if gravity < 0.25:
        cost += 34*7
        note += "Low Grav (MGC), "
    if temperature > 75:
        cost += 680
        note += "High Temp (TSH), "
    if temperature < -25:
        cost += 468
        note += "Low Temp (INS), "
    if pressure > 2.0:
        cost += 272
        note += "High Press (HSE), "
    if pressure < 0.25:
        cost += 23
        note += "Low Press (SEA), "
    if not(surface):
        cost += 324
        note += "Gas (AEF), "

    if planetDf["HasChamberOfCommerce"]:
        cogc = str(planetDf["COGCPrograms"][-1]["ProgramType"])
        cogc = to_camel_case(cogc.replace("ADVERTISING_", ""))
        note = "COGC: " + cogc

    if cost < 2800:
        print("     " + planetDf["PlanetName"] + "         HYF Base Cost: " + str(cost) + "k      "+ note)

def SearchNear(searchSystemNode, title, rangeN):

    print("***************************")
    print("*  Searching near " + title + " (" + searchSystemNode + ")")
    print("***************************")
    # within N jumps of ANT
    def nearSystem(systemName):
        pathLen = nx.shortest_path_length(fullGraph, searchSystemNode, systemName)
        # print("Distance from " + systemName + " is " + str(pathLen))
        return pathLen < rangeN

    g2 = FilterByFunction(nearSystem)

    # // color as distance from nascent:
    filteredValue = dict(nx.single_source_shortest_path_length(g2, searchSystemNode))
    print(searchSystemNode)
    # print(filteredValue)
    return g2

def SortAndEvalPlanets(g2, searchSystemNode):
    def funcS(x):
        return nx.shortest_path_length(g2, searchSystemNode, x)
    
    sortedNodes = list(g2.nodes)
    sortedNodes.sort(key=funcS)
    # print(sortedNodes)

    for systemName in sortedNodes:
        print()
        print(f"{systemName}      Distance {nx.shortest_path_length(g2, searchSystemNode, systemName)}")
        systemDf = FindSystem(systemName)
        for planetDf in FindPlanetsIn(systemDf):
            # evaluate planet suitability:
            EvaluatePlanet(planetDf)


g2 = SearchNear("QJ-149", "Nascent", 3) #Nascent
SortAndEvalPlanets(g2, "QJ-149")

g2 = SearchNear("SE-110", "Ice Station Alpha", 3)
SortAndEvalPlanets(g2, "SE-110")

g2 = SearchNear("KW-688", "Etherwind", 3)
SortAndEvalPlanets(g2, "KW-688")


        
        
        
# PlotGraph(g2)